# Test Notebook: Stitching Utils Module

This notebook tests the `stitching_utils` module for combining segment alignments.

**Features tested:**
1. Module imports and structure
2. Data classes (SegmentAlignment, StitchingResult)
3. LIS Stitcher
4. Edit Distance Stitcher
5. Diff Stitcher
6. High-level API

**Stitching Methods:**
- `LISStitcher`: Uses Longest Increasing Subsequence - O(N log N)
- `EditDistanceStitcher`: Uses Levenshtein distance - O(N*M)
- `DiffStitcher`: Uses Python difflib (Gentle style)

## Setup

In [ ]:
# Install dependencies if needed
# ! pip install git+https://github.com/huangruizhe/lis.git

In [ ]:
# =============================================================================
# Setup: Configure Imports
# =============================================================================

import sys
import os
from pathlib import Path

# ===== CONFIGURATION =====
GITHUB_REPO = "https://github.com/huangruizhe/torchaudio_aligner.git"
BRANCH = "dev"  # Use 'dev' for testing, 'main' for stable
# =========================

# Test result tracking
test_results = {}

def setup_imports():
    """Setup Python path for imports based on environment."""
    
    IN_COLAB = 'google.colab' in sys.modules
    
    if IN_COLAB:
        repo_path = '/content/torchaudio_aligner'
        src_path = f'{repo_path}/src'
        
        if not os.path.exists(repo_path):
            print(f"Cloning repository (branch: {BRANCH})...")
            os.system(f'git clone -b {BRANCH} {GITHUB_REPO} {repo_path}')
            print("Repository cloned")
        else:
            print(f"Updating repository (branch: {BRANCH})...")
            os.system(f'cd {repo_path} && git fetch origin && git checkout {BRANCH} && git pull origin {BRANCH}')
            print("Repository updated")
    else:
        possible_paths = [
            Path(".").absolute().parent / "src",
            Path(".").absolute() / "src",
        ]
        
        src_path = None
        for p in possible_paths:
            if p.exists() and (p / "stitching_utils").exists():
                src_path = str(p.absolute())
                break
        
        if src_path is None:
            raise FileNotFoundError("src directory not found")
        
        print(f"Running locally from: {src_path}")
    
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
    
    return src_path

src_path = setup_imports()

import logging
logging.basicConfig(level=logging.INFO)

# Check dependencies
print()
print("=" * 60)
print("Checking dependencies...")
print("=" * 60)

# Check lis
try:
    import lis
    print("\u2705 lis library available")
    LIS_AVAILABLE = True
except ImportError:
    print("\u26a0\ufe0f lis not available - LIS tests will be limited")
    print("   Install with: pip install git+https://github.com/huangruizhe/lis.git")
    LIS_AVAILABLE = False

## Test 1: Module Imports and Structure

In [ ]:
print("=" * 60)
print("Test 1: Module Imports and Structure")
print("=" * 60)

try:
    from stitching_utils import (
        # Base classes
        StitcherBackend,
        StitchingConfig,
        SegmentAlignment,
        # Stitchers
        LISStitcher,
        EditDistanceStitcher,
        DiffStitcher,
        # API
        stitch_alignments,
        get_stitcher,
        list_methods,
    )
    
    print("\U0001F4E6 Imports successful!")
    
    # List available methods
    methods = list_methods()
    print("\n\U0001F527 Available stitching methods:")
    for name, info in methods.items():
        print(f"   \u2705 {name}: {info['description']}")
        print(f"      Requires reference: {info['requires_reference']}")
        print(f"      Best for: {info['best_for']}")
    
    test_results["Test 1"] = "\u2705 PASSED"
    print(f"\n\u2705 Test 1 PASSED - Module imports successful")
except Exception as e:
    test_results["Test 1"] = "\u274c FAILED"
    print(f"\n\u274c Test 1 FAILED: {e}")
    import traceback
    traceback.print_exc()

## Test 2: Data Classes

In [ ]:
print("=" * 60)
print("Test 2: Data Classes (SegmentAlignment, StitchingConfig)")
print("=" * 60)

try:
    from stitching_utils.base import (
        AlignedToken,
        SegmentAlignment,
        StitchingResult,
        StitchingConfig,
    )
    
    # Test AlignedToken
    print("\n\U0001F4DD AlignedToken:")
    token = AlignedToken(
        token_id=5,
        timestamp=100,
        score=0.95,
        attr={"wid": 0, "tk": "h"},
    )
    print(f"   \u2022 token_id: {token.token_id}")
    print(f"   \u2022 timestamp: {token.timestamp}")
    print(f"   \u2022 score: {token.score}")
    print(f"   \u2022 has_word_index: {token.has_word_index}")
    
    # Test SegmentAlignment
    print("\n\U0001F4CB SegmentAlignment:")
    tokens = [
        AlignedToken(5, 100, 0.9, {"wid": 0, "tk": "h"}),
        AlignedToken(4, 105, 0.85, {"tk": "e"}),
        AlignedToken(11, 110, 0.88, {"tk": "l"}),
        AlignedToken(11, 115, 0.92, {"tk": "l"}),
        AlignedToken(14, 120, 0.87, {"tk": "o"}),
        AlignedToken(22, 130, 0.91, {"wid": 1, "tk": "w"}),
    ]
    segment = SegmentAlignment(
        tokens=tokens,
        segment_index=0,
        frame_offset=0,
    )
    print(f"   \u2022 Num tokens: {len(segment)}")
    print(f"   \u2022 Word indices: {segment.get_word_indices()}")
    print(f"   \u2022 Segment index: {segment.segment_index}")
    
    # Test StitchingConfig
    print("\n\u2699\ufe0f StitchingConfig:")
    config = StitchingConfig(
        method="lis",
        neighborhood_size=5,
        outlier_threshold=50,
    )
    print(f"   \u2022 Method: {config.method}")
    print(f"   \u2022 Neighborhood size: {config.neighborhood_size}")
    print(f"   \u2022 Outlier threshold: {config.outlier_threshold}")
    
    test_results["Test 2"] = "\u2705 PASSED"
    print(f"\n\u2705 Test 2 PASSED - Data classes work correctly")
except Exception as e:
    test_results["Test 2"] = "\u274c FAILED"
    print(f"\n\u274c Test 2 FAILED: {e}")
    import traceback
    traceback.print_exc()

## Test 3: LIS Stitcher

In [ ]:
print("=" * 60)
print("Test 3: LIS Stitcher")
print("=" * 60)

if not LIS_AVAILABLE:
    test_results["Test 3"] = "\u23ed\ufe0f SKIPPED"
    print("\u23ed\ufe0f Test 3 SKIPPED - lis library not available")
    print("   Install with: pip install git+https://github.com/huangruizhe/lis.git")
else:
    try:
        from stitching_utils import LISStitcher, StitchingConfig
        from stitching_utils.base import AlignedToken, SegmentAlignment
        from stitching_utils.lis import (
            compute_lis,
            remove_outliers,
            remove_isolated_words,
            find_unaligned_regions,
        )
        
        # Test LIS computation
        print("\n\U0001F4C8 LIS computation:")
        # Simulating word indices from overlapping segments
        word_indices = [1, 5, 2, 6, 3, 7, 4, 8, 9, 10, 11, 15, 12, 16, 13]
        print(f"   \u2022 Input: {word_indices}")
        
        lis_result = compute_lis(word_indices)
        print(f"   \u2022 LIS: {lis_result}")
        print(f"   \u2022 LIS length: {len(lis_result)}")
        
        # Verify LIS is increasing
        is_increasing = all(lis_result[i] < lis_result[i+1] for i in range(len(lis_result)-1))
        print(f"   \u2022 Is strictly increasing: {'\u2705' if is_increasing else '\u274c'}")
        
        # Test outlier removal
        print(f"\n\U0001F50D Outlier removal:")
        with_outliers = [5, 100, 10, 15, 20, 25, 30, 35, 200]
        cleaned = remove_outliers(with_outliers, scan_range=3, outlier_threshold=50)
        print(f"   \u2022 Input: {with_outliers}")
        print(f"   \u2022 Cleaned: {cleaned}")
        
        # Test full stitcher
        print(f"\n\U0001F9F5 LIS Stitcher:")
        
        # Create mock segment alignments
        seg1_tokens = [
            AlignedToken(1, 10, 0.9, {"wid": 0}),
            AlignedToken(2, 20, 0.85, {"wid": 1}),
            AlignedToken(3, 30, 0.88, {"wid": 2}),
        ]
        seg2_tokens = [
            AlignedToken(2, 25, 0.87, {"wid": 1}),  # Overlap
            AlignedToken(3, 35, 0.91, {"wid": 2}),  # Overlap
            AlignedToken(4, 45, 0.89, {"wid": 3}),
            AlignedToken(5, 55, 0.92, {"wid": 4}),
        ]
        
        segments = [
            SegmentAlignment(seg1_tokens, segment_index=0),
            SegmentAlignment(seg2_tokens, segment_index=1),
        ]
        
        stitcher = LISStitcher()
        result = stitcher.stitch(segments)
        
        print(f"   \u2022 Input segments: {len(segments)}")
        print(f"   \u2022 Output tokens: {result.num_aligned_tokens}")
        print(f"   \u2022 Unaligned regions: {result.unaligned_regions}")
        print(f"   \u2022 Word indices: {result.get_word_indices()}")
        
        test_results["Test 3"] = "\u2705 PASSED"
        print(f"\n\u2705 Test 3 PASSED - LIS Stitcher works correctly")
    except Exception as e:
        test_results["Test 3"] = "\u274c FAILED"
        print(f"\n\u274c Test 3 FAILED: {e}")
        import traceback
        traceback.print_exc()

## Test 4: Edit Distance Stitcher

In [ ]:
print("=" * 60)
print("Test 4: Edit Distance Stitcher")
print("=" * 60)

try:
    from stitching_utils import EditDistanceStitcher
    from stitching_utils.base import AlignedToken, SegmentAlignment
    
    # Create mock segment with word attributes
    seg_tokens = [
        AlignedToken(1, 10, 0.9, {"word": "hello"}),
        AlignedToken(2, 50, 0.85, {"word": "wrld"}),  # Typo
        AlignedToken(3, 90, 0.88, {"word": "test"}),
    ]
    segments = [SegmentAlignment(seg_tokens, segment_index=0)]
    
    reference = ["hello", "world", "test"]
    
    print("\n\U0001F4DD Edit Distance alignment:")
    print(f"   \u2022 Predicted: {[t.attr['word'] for t in seg_tokens]}")
    print(f"   \u2022 Reference: {reference}")
    
    stitcher = EditDistanceStitcher()
    result = stitcher.stitch(segments, reference_words=reference)
    
    print(f"\n\U0001F4CA Results:")
    print(f"   \u2022 Aligned tokens: {result.num_aligned_tokens}")
    print(f"   \u2022 Edit distance: {result.metadata.get('edit_distance', 'N/A')}")
    
    for token in result.tokens:
        print(f"   \u2022 Word '{token.attr.get('word', '?')}' at frame {token.timestamp}")
    
    test_results["Test 4"] = "\u2705 PASSED"
    print(f"\n\u2705 Test 4 PASSED - Edit Distance Stitcher works correctly")
except Exception as e:
    test_results["Test 4"] = "\u274c FAILED"
    print(f"\n\u274c Test 4 FAILED: {e}")
    import traceback
    traceback.print_exc()

## Test 5: Diff Stitcher

In [ ]:
print("=" * 60)
print("Test 5: Diff Stitcher (Gentle style)")
print("=" * 60)

try:
    from stitching_utils import DiffStitcher
    from stitching_utils.base import AlignedToken, SegmentAlignment
    from stitching_utils.diff import word_diff
    
    # Test word_diff utility (Gentle's approach)
    print("\n📝 Testing word_diff (Gentle's approach):")
    pred = ["the", "quick", "brwn", "fox"]  # Note: 'brwn' is a typo
    ref = ["the", "quick", "brown", "fox", "jumps"]
    
    print(f"   • Predicted: {pred}")
    print(f"   • Reference: {ref}")
    print("   • Operations:")
    for op, a_idx, b_idx in word_diff(pred, ref):
        if op == 'equal':
            print(f"      ✓ {op}: pred[{a_idx}]='{pred[a_idx]}' == ref[{b_idx}]='{ref[b_idx]}'")
        elif op == 'replace':
            print(f"      ~ {op}: pred[{a_idx}]='{pred[a_idx]}' -> ref[{b_idx}]='{ref[b_idx]}'")
        elif op == 'delete':
            print(f"      - {op}: pred[{a_idx}]='{pred[a_idx]}' (not in ref)")
        elif op == 'insert':
            print(f"      + {op}: ref[{b_idx}]='{ref[b_idx]}' (not in pred)")
    
    # Create mock segment
    seg_tokens = [
        AlignedToken(1, 10, 0.9, {"word": "the"}),
        AlignedToken(2, 30, 0.85, {"word": "quick"}),
        AlignedToken(3, 60, 0.88, {"word": "brown"}),
        AlignedToken(4, 90, 0.91, {"word": "fox"}),
    ]
    segments = [SegmentAlignment(seg_tokens, segment_index=0)]
    
    reference = ["the", "quick", "brown", "fox", "jumps"]
    
    print("\n🔗 Diff Stitcher alignment:")
    print(f"   • Predicted: {[t.attr['word'] for t in seg_tokens]}")
    print(f"   • Reference: {reference}")
    
    stitcher = DiffStitcher()
    result = stitcher.stitch(segments, reference_words=reference)
    
    print(f"\n📊 Results:")
    print(f"   • Aligned tokens: {result.num_aligned_tokens}")
    print(f"   • Similarity ratio: {result.metadata.get('ratio', 'N/A'):.2%}")
    stats = result.metadata.get('stats', {})
    print(f"   • Stats: equal={stats.get('equal', 0)}, replace={stats.get('replace', 0)}, insert={stats.get('insert', 0)}, delete={stats.get('delete', 0)}")
    print(f"   • Unaligned regions: {result.unaligned_regions}")
    
    for token in result.tokens:
        case = token.attr.get('case', 'unknown')
        print(f"   • Word '{token.attr.get('word', '?')}' (ref idx {token.attr.get('wid', '?')}) at frame {token.timestamp} [{case}]")
    
    test_results["Test 5"] = "✅ PASSED"
    print(f"\n✅ Test 5 PASSED - Diff Stitcher (Gentle style) works correctly")
except Exception as e:
    test_results["Test 5"] = "❌ FAILED"
    print(f"\n❌ Test 5 FAILED: {e}")
    import traceback
    traceback.print_exc()

## Test 6: High-level API

In [ ]:
print("=" * 60)
print("Test 6: High-level API (stitch_alignments)")
print("=" * 60)

try:
    from stitching_utils import stitch_alignments, get_stitcher
    from stitching_utils.base import AlignedToken, SegmentAlignment
    
    # Create mock overlapping segments
    seg1 = SegmentAlignment(
        tokens=[
            AlignedToken(1, 10, 0.9, {"wid": 0, "word": "hello"}),
            AlignedToken(2, 50, 0.85, {"wid": 1, "word": "world"}),
        ],
        segment_index=0,
    )
    seg2 = SegmentAlignment(
        tokens=[
            AlignedToken(2, 55, 0.87, {"wid": 1, "word": "world"}),  # Overlap
            AlignedToken(3, 90, 0.91, {"wid": 2, "word": "test"}),
        ],
        segment_index=1,
    )
    segments = [seg1, seg2]
    
    print("\n\U0001F527 Testing get_stitcher():")
    for method in ["lis", "edit_distance", "diff"]:
        try:
            stitcher = get_stitcher(method)
            print(f"   \u2705 {method}: {stitcher.name}")
        except Exception as e:
            print(f"   \u26a0\ufe0f {method}: {e}")
    
    print("\n\U0001F9F5 Testing stitch_alignments():")
    
    # Test with edit_distance (doesn't need lis library)
    result = stitch_alignments(
        segments,
        method="edit_distance",
        reference_words=["hello", "world", "test"],
    )
    print(f"   \u2022 edit_distance: {result.num_aligned_tokens} tokens aligned")
    
    # Test with diff
    result = stitch_alignments(
        segments,
        method="diff",
        reference_words=["hello", "world", "test"],
    )
    print(f"   \u2022 diff: {result.num_aligned_tokens} tokens aligned")
    
    # Test with LIS if available
    if LIS_AVAILABLE:
        result = stitch_alignments(segments, method="lis")
        print(f"   \u2022 lis: {result.num_aligned_tokens} tokens aligned")
    else:
        print(f"   \u23ed\ufe0f lis: skipped (library not available)")
    
    test_results["Test 6"] = "\u2705 PASSED"
    print(f"\n\u2705 Test 6 PASSED - High-level API works correctly")
except Exception as e:
    test_results["Test 6"] = "\u274c FAILED"
    print(f"\n\u274c Test 6 FAILED: {e}")
    import traceback
    traceback.print_exc()

## Test Summary

In [ ]:
print("=" * 60)
print("\U0001F4CB TEST RESULTS SUMMARY")
print("=" * 60)

# Display test results
print("\n" + "-" * 40)
for test_name, result in test_results.items():
    print(f"  {result}  {test_name}")
print("-" * 40)

# Count results
passed = sum(1 for r in test_results.values() if "\u2705" in r)
failed = sum(1 for r in test_results.values() if "\u274c" in r)
skipped = sum(1 for r in test_results.values() if "\u23ed" in r)
total = len(test_results)

print(f"\n  Total: {total} tests")
print(f"  \u2705 Passed:  {passed}")
if skipped > 0:
    print(f"  \u23ed\ufe0f Skipped: {skipped}")
if failed > 0:
    print(f"  \u274c Failed:  {failed}")

print("\n" + "=" * 60)
if failed == 0:
    print("\U0001F389 All tests passed (or skipped due to optional dependencies)!")
else:
    print(f"\u26a0\ufe0f {failed} test(s) failed - please check above for details")
print("=" * 60)

print("\n\U0001F4E6 To enable all tests, install:")
print("   pip install git+https://github.com/huangruizhe/lis.git")

print("\n\U0001F3D7\ufe0f Architecture note:")
print("   Stitching utils combines segment alignments from the alignment module.")
print("   Available methods:")
print("   \u2022 LIS: Best for WFST with word index output labels")
print("   \u2022 Edit Distance: Best for ASR/MFA output")
print("   \u2022 Diff: Best for mostly accurate transcripts")